# Markov Chain Simulation of State Dependent SSQ
Consider a single-server Markovian queue with state dependent arrivals and service. You are free to choose the λi and μi rates, but make sure they are different for different i values. Set the buffer size at k = 200. Solve the steady-state equations using the method of successive substitutions and using a standard method (e.g. using Cramer’s rule). Compare the results and the computation time. Then, obtain the blocking probability by simulation and compare the results and the running times with the equivalent blocking probability results and running times obtained by solving the state equations.

In [9]:
import numpy as np
import random as random
import scipy.stats as stats

## Simulation of State Dependent SSQ


Based on the state transition diagram, a pseudo code of Markov Chain simulation is given:

Initialization: Q (QueueSize) = 0; Na (Number of Arrivals) = 0, Nb (Number of Blocks) = 0, k (Buffer Size).

1. If R(01) ≤ λ/(λ + Qμ)

        Na = Na + 1;

             if Q < k - 1: Q = Q + 1;

             else: Nb = Nb + 1;

        else: Q = Q - 1.

2. If Na < MAXNa go to 1;

    else, print Bp = Nb/Na.

In [10]:
def state_dependent_ssq_mk_simulation(lam, mu, k, total_arrivals):
    queue_size = 0
    num_arrivals = 0
    num_blocked = 0

    while num_arrivals < total_arrivals:
        # arrival
        if random.random() <= lam[queue_size] / (lam[queue_size] + int(queue_size > 0) * mu[queue_size]):
            num_arrivals += 1
            # within the buffer size
            if queue_size < k - 1:
                queue_size += 1
            # exceed the buffer size
            else:
                num_blocked += 1
        # departure
        else:
            queue_size -= 1
    blocking_probability = num_blocked / num_arrivals
    return blocking_probability

## Theoretical Blocking Probability
The steady-state equation is $\pi_{i-1}\lambda_{i-1}=\pi_i\mu_i,i=1,2,3,\ldots,k$.

The normalizing equation is $\sum_{i=0}^k\pi_i=1$.

Given a set of lambda and mu, all the pi can be calculated.

The results are
$$\pi_0=\frac{1}{1+\sum_{i=1}^k\frac{\prod_{j=0}^{i-1}\lambda_j}{\prod_{m=1}^i\mu_m}}$$
$$\pi_k=\pi_0\frac{\prod_{j=1}^k\mu_{_j}}{\prod_{i=0}^{k-1}\lambda_i}$$



In [11]:
def theoretical_blocking_probability(lam, mu, k):
    pi = np.zeros(k)
    # calculate pi(0)
    temp = 1
    for i in range(1, k):
        temp += np.prod(lam[0 : i]) / np.prod(mu[1 : i + 1])
    pi[0] = 1 / temp

    #calculate pi(1) ~ pi(k - 1)
    for i in range(1, k):
        pi[i] = pi[0] * np.prod(lam[0 : i]) / np.prod(mu[1 : i + 1])

    blocking_probability = pi[k - 1] * lam[k - 1] / np.sum(pi * lam)
    return blocking_probability

## Blocking Probability
Parameters: k = 25, the arrival rate is a set of random variables ranging from 2 to 10, and the service rate is a set of random variables ranging from 1 to 8.

The blocking probability of the Markov Chain is very close to the theoretical value. The theoretical value also lies in the confidence interval.

However, when I change the k to 200, the result is wrong because the value overflows.

In [24]:
k = 25
lam = np.random.randint(2, 10, k)
mu = np.random.randint(1, 8, k)
loop_time = 50

mk_blocking_prob = np.zeros(loop_time)
for i in range(loop_time):
    mk_blocking_prob[i] = state_dependent_ssq_mk_simulation(lam, mu, k, 100000)

mk_confidence_interval = stats.norm.interval(alpha = 0.95,
                                             loc = np.mean(mk_blocking_prob),
                                             scale = stats.sem(mk_blocking_prob))

print("Mean Blocking Probability (Markov Chain):", np.mean(mk_blocking_prob))
print("Confidence Interval of Blocking Probability (Markov Chain):", mk_confidence_interval)
print("Theoretical Blocking Probability:", theoretical_blocking_probability(lam, mu, k))

Mean Blocking Probability (Markov Chain): 0.014392999999999998
Confidence Interval of Blocking Probability (Markov Chain): (0.014205870198384473, 0.014580129801615523)
Theoretical Blocking Probability: 0.014275629703299124
